# Feature engineering, xgboost
https://www.kaggle.com/dlarionov/feature-engineering-xgboost


-　DateNum作成 'date_block_num',→YYYYMMで代用
- 祝日フラグ
- month
- days
- アイテムカテゴリ（デフォルト)-> カテゴリCategory、ジャンルType
- 月の売り上げ
- 月の売り上げラグ 1,2,3,6,12
- アイテムあたりの「週Or日」平均販売個数ラグ1，2，3，6，12　　'date_item_avg_item_cnt_lag_1',
- ショップあたり「週Or日」平均販売個数ラグ1，2，3，6，12　　'date_shop_avg_item_cnt_lag_1',
- アイテムカテゴリあたり「週Or日」平均販売個数ラグ1，2，3，'date_cat_avg_item_cnt_lag_1',
- カテゴリあたり「週Or日」平均販売個数ラグ1，2，3，'date_cat_avg_item_cnt_lag_1',
- カテゴリ/ショップあたり[月] 平均販売個数ラグ1，2，3，
- ジャンル　あたり[月] 平均販売個数ラグ1，2，3，
- ジャンル/ショップ　あたり[月] 平均販売個数ラグ1，2，3，
- ASP　月当たり
- 価格下落率
- 最終販売日
- 初販売日
- ショップあたりの最終販売日
- ショップあたりの初販売日
    'delta_price_lag',


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
category_names = pd.read_csv('category_names.csv')
category_names.columns = ['ItemCategoryId', 'ItemCategoryName']

item_categories = pd.read_csv('item_categories.csv')
item_categories.columns = ['ItemId', 'ItemCategoryId']

sales_history = pd.read_csv('sales_history.csv')
sales_history.columns = ['Date', 'StoreId', 'ItemId', 'ItemPrice', 'SoldNum']
sales_history['Date'] = pd.to_datetime(sales_history['Date'])

test = pd.read_csv('test.csv')
test.columns = ['index', 'ItemId', 'StoreId']
#item_categories['ItemCategoryId'].unique()
#category_names.query('ItemCategoryName.str.contains("")')
'''170品目のリスト'''
test_item = test['ItemId'].unique()
test_item = pd.DataFrame({'ItemId':test_item})
#test_item
'''カテゴリ名と連結'''
ttest = pd.merge(test,item_categories, on='ItemId')
ttest = pd.merge(ttest, category_names,on='ItemCategoryId')
print('予測データの品目数: ',ttest['ItemId'].nunique(),'\n',
      '訓練データの品目数: ',sales_history['ItemId'].nunique(),'\n',
      '予測データのカテゴリ数: ',item_categories['ItemCategoryId'].nunique(),'\n',
      '訓練データのカテゴリ数: ',ttest['ItemCategoryId'].nunique(),'\n',
     '売上の総数: ',sales_history['SoldNum'].sum(), '\n',
     '')

予測データの品目数:  170 
 訓練データの品目数:  9040 
 予測データのカテゴリ数:  26 
 訓練データのカテゴリ数:  26 
 売上の総数:  1262869.0 
 


# 休日フラグ  sales_total.csvとして保存
https://upura.hatenablog.com/entry/2018/12/21/070000

In [3]:
sales_total = pd.read_csv('sales_total.csv')
'''
from datetime import date,timedelta
import jpholiday
%time
sales_total['is_holiday'] = sales_total['Date'].map(jpholiday.is_holiday).astype(int)'''

"\nfrom datetime import date,timedelta\nimport jpholiday\n%time\nsales_total['is_holiday'] = sales_total['Date'].map(jpholiday.is_holiday).astype(int)"

In [4]:
sales_total

,Date,StoreId,ItemId,ItemPrice,SoldNum,ItemCategoryId,CatName,TypeName,Category,Type,is_holiday
0,2018-01-01,9,1000001,420,1.0,100,映画,DVD,5,1,1
1,2018-01-01,6,1000001,420,1.0,100,映画,DVD,5,1,1
2,2018-01-01,10,1000001,420,1.0,100,映画,DVD,5,1,1
3,2018-01-02,10,1000001,420,2.0,100,映画,DVD,5,1,0
4,2018-01-02,9,1000001,420,1.0,100,映画,DVD,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1119565,2019-10-20,6,3300319,3400,1.0,330,音楽,レコード,7,17,0
1119566,2019-10-20,0,3300319,3400,1.0,330,音楽,レコード,7,17,0
1119567,2019-10-25,0,3300323,9350,1.0,330,音楽,レコード,7,17,0
1119568,2019-10-28,1,3300325,2120,1.0,330,音楽,レコード,7,17,0


In [5]:
#sales_total.to_csv('sales_total.csv',index=False)

# 年月にグループ化

In [6]:
sales_total['Date'] = pd.to_datetime(sales_total['Date'])
sales_total['YYYYMM'] = sales_total['Date'].dt.strftime('%Y%m')

In [7]:
sales_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119570 entries, 0 to 1119569
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   Date            1119570 non-null  datetime64[ns]
 1   StoreId         1119570 non-null  int64         
 2   ItemId          1119570 non-null  int64         
 3   ItemPrice       1119570 non-null  int64         
 4   SoldNum         1119570 non-null  float64       
 5   ItemCategoryId  1119570 non-null  int64         
 6   CatName         1119570 non-null  object        
 7   TypeName        1119570 non-null  object        
 8   Category        1119570 non-null  int64         
 9   Type            1119570 non-null  int64         
 10  is_holiday      1119570 non-null  int64         
 11  YYYYMM          1119570 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(7), object(3)
memory usage: 102.5+ MB


# 単価→ASP , 売上個数集計

In [8]:
monthly_sales = sales_total.groupby([
    'YYYYMM','StoreId','ItemId','ItemCategoryId','Category',
    'Type']).agg({'ItemPrice':'mean','SoldNum':'sum','is_holiday':'sum'}).reset_index()

In [9]:
monthly_sales

,YYYYMM,StoreId,ItemId,ItemCategoryId,Category,Type,ItemPrice,SoldNum,is_holiday
0,201801,0,1000001,100,5,1,420.0,6.0,0
1,201801,0,1000002,100,5,1,250.0,2.0,0
2,201801,0,1000004,100,5,1,120.0,1.0,0
3,201801,0,1000005,100,5,1,220.0,8.0,0
4,201801,0,1000006,100,5,1,190.0,2.0,0
...,...,...,...,...,...,...,...,...,...
495129,201910,17,3400005,340,4,9,1190.0,1.0,0
495130,201910,17,3400006,340,4,9,2700.0,1.0,0
495131,201910,17,3400012,340,4,9,680.0,1.0,0
495132,201910,17,3400016,340,4,9,250.0,1.0,0


# 全データから

In [13]:
sales_history = pd.merge(sales_history,test_item,on='ItemId')
sales_history

,Date,StoreId,ItemId,ItemPrice,SoldNum,ItemCategoryId,YYYYMM
0,2018-01-01,9,1000001,420,1.0,100,201801
1,2018-01-01,6,1000001,420,1.0,100,201801
2,2018-01-01,10,1000001,420,1.0,100,201801
3,2018-01-02,10,1000001,420,2.0,100,201801
4,2018-01-02,9,1000001,420,1.0,100,201801
...,...,...,...,...,...,...,...
48216,2019-10-31,7,1002582,680,1.0,100,201910
48217,2019-10-31,6,1002582,680,1.0,100,201910
48218,2019-10-31,3,1002582,680,1.0,100,201910
48219,2019-10-31,15,1002582,460,1.0,100,201910


In [14]:
monthly_sales = sales_history.drop('Date',axis=1)
monthly_sales

,StoreId,ItemId,ItemPrice,SoldNum,ItemCategoryId,YYYYMM
0,9,1000001,420,1.0,100,201801
1,6,1000001,420,1.0,100,201801
2,10,1000001,420,1.0,100,201801
3,10,1000001,420,2.0,100,201801
4,9,1000001,420,1.0,100,201801
...,...,...,...,...,...,...
48216,7,1002582,680,1.0,100,201910
48217,6,1002582,680,1.0,100,201910
48218,3,1002582,680,1.0,100,201910
48219,15,1002582,460,1.0,100,201910


In [15]:
monthly_sales.groupby(['StoreId','ItemId',
                       'ItemPrice','ItemCategoryId']).agg({'ItemPrice':'mean',
                                                          'SoldNum':sum})

ItemPrice  SoldNum
StoreId ItemId  ItemPrice ItemCategoryId                    
0       1000001 230       100                   230      1.0
                250       100                   250      3.0
                420       100                   420     26.0
        1000002 250       100                   250     10.0
        1000003 170       100                   170      1.0
...                                             ...      ...
17      3400003 830       340                   830      1.0
                1020      340                  1020      1.0
                1100      340                  1100      2.0
                1190      340                  1190     20.0
        3500001 420       350                   420      1.0

[5580 rows x 2 columns]

ラグ関数

In [16]:
def lag_feature(df,lags,col):
    tmp = df[['Date','StoreId','ItemId',
              'ItemCategoryId',col]]
    for i in lags:
        shfted = tmp.copy()
        shifted.columns = ['Date','StoreId','ItemId',
              'ItemCategoryId',col+'_lag_'+str(i)]
        shifted['Date']